### 1. Setup and Import Libraries

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import os


In [ ]:
# Define paths to local files
file_paths = {
    "TSLA": "data/TSLA_proccessed_data.csv",
    "SPY": "data/SPY_proccessed_data.csv",
    "BND": "data/BND_proccessed_data.csv"
}

# Load data function
def load_data(file_path):
    return pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')

# Load all datasets into a dictionary
data_dict = {ticker: load_data(path) for ticker, path in file_paths.items()}
